In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
%matplotlib inline

In [3]:
cd ..\experiments\data\human\0.991

C:\Users\piichan\Documents\Berkeley\CoCoSci Lab\mcrl\mcrl\experiments\data\human\0.991


In [22]:
df=pd.read_csv('trials.csv', sep=',',header=None)
df


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,NaN,pid,info_cost,PR_type,message,trial_index,trial_i,delays,score,n_click,clicks,click_times,path,action_times
1,0,0,1.0,none,none,1,NaN,[3],12.0,16,"[5, 6, 7, 8, 9, 10, 12, 11, 1, 2, 3, 4, 13, 14...","[2692, 3813, 4808, 7601, 13713, 15045, 15845, ...","[1, 5, 6, 7]","[71892, 77817, 81863]"
2,1,0,1.0,none,none,2,NaN,[3],26.0,2,"[5, 9]","[3801, 5825]","[1, 9, 10, 12]","[7534, 21008, 22827]"
3,2,0,1.0,none,none,3,NaN,[3],26.0,2,"[5, 6]","[2081, 3815]","[1, 5, 6, 8]","[5649, 14086, 16738]"
4,3,0,1.0,none,none,4,NaN,[3],-8.0,4,"[5, 13, 9, 1]","[1609, 2946, 4339, 5745]","[1, 5, 6, 7]","[8788, 15820, 17077]"
5,4,0,1.0,none,none,5,NaN,[3],19.0,1,[5],[3112],"[1, 5, 6, 7]","[5477, 9516, 14710]"
6,5,0,1.0,none,none,6,NaN,[3],4.0,0,[],[],"[1, 5, 6, 8]","[3077, 7200, 10754]"
7,6,0,1.0,none,none,7,NaN,[3],28.0,0,[],[],"[1, 9, 10, 12]","[2198, 6520, 8274]"
8,7,0,1.0,none,none,8,NaN,[3],-4.0,0,[],[],"[1, 1, 2, 4]","[3118, 6943, 8352]"
9,8,0,1.0,none,none,9,NaN,[3],4.0,0,[],[],"[1, 5, 6, 7]","[2954, 6727, 8846]"
